In [1]:
%cd ../

/home/changsun/minigpt4


In [2]:
from typing import TYPE_CHECKING, cast
import PIL
import bentoml
from pydantic import BaseModel
from bentoml.io import JSON, Multipart, Text, NumpyNdarray, Image
import numpy as np
import os
import torch
#import torchvision.transforms.functional as TF
#import torchvision.transforms as T
#import PIL

#if TYPE_CHECKING:
    #from PIL.Image import Image


TARGET_IMAGE_SIZE = 256

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # for debugging purpose
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # for debugging purpose

rrg_runner = bentoml.models.get("brain-mri-rrg:latest").to_runner()

svc = bentoml.Service(name="brain-mri-rrg", runners=[rrg_runner])


class TextResponseDTO(BaseModel):
    answer: str

    class Config:
        extra = "forbid"
        arbitrary_types_allowed = True

# ImageToText
@svc.api(
    input=Multipart(query=Text(), img=NumpyNdarray()),
    output=JSON(pydantic_model=TextResponseDTO),
)

async def reportgen(query: str, img: np.ndarray) -> TextResponseDTO:
    
    
    if img.ndim != 4:
        raise ValueError(f"Incorrect number of dimensions for MRI image processing")

    m = img.shape[2]
    c = img.shape[3]
    m
    if m < TARGET_IMAGE_SIZE:
        raise ValueError(f"min dim for image {m} < {TARGET_IMAGE_SIZE}")
    if c != 3:
        raise ValueError(f"Incorrect number of channels for MRI image processing")
    
    ###Prompt Generation
    img_tag = '<Img><ImageHere></Img>'
    prompt = img_tag + query
    model_input = (img, prompt)

    ###Report Generation
    answer = await rrg_runner.async_run(model_input, max_new_tokens=200)
    return TextResponseDTO(
        answer=answer,
    )




In [3]:
for runner in svc.runners:
    runner.init_local()

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.
/home/changsun/anaconda3/envs/minigpt4/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/changsun/anaconda3/envs/minigpt4/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


In [4]:
inp = {
    "query": " This is a volumetric brain MRI scan showing abnormalities. Diagnose this patient with this MRI scan.",
    "img": np.load('/home/changsun/minigpt4/bento/TCGA_CS_4941_19960909.npy'), 
}
result = await svc.apis["reportgen"].func(**inp)

<Img><ImageHere></Img> This is a volumetric brain MRI scan showing abnormalities. Diagnose this patient with this MRI scan.
['<Img>', '</Img> This is a volumetric brain MRI scan showing abnormalities. Diagnose this patient with this MRI scan.']


/home/changsun/anaconda3/envs/minigpt4/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/changsun/anaconda3/envs/minigpt4/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [6]:
print(result)

answer='the mri reveals an oligodendroglioma a grade 3 neoplasm located in the left supratentorial frontal lobe. this denotes a high-grade malignant tumor typical of aggressive growth patterns. the tumors position in the left frontal lobe suggests potential impact on cognitive functions and personality individuals typically seen in patients with such a diagnosis.'
